# RAG + Zero Prompting + Tools + Chain of Thought

Este notebook demonstra um exemplo completo combinando:
- **RAG**: Recuperação de informações de documentos
- **Zero Prompting**: Sem exemplos no prompt
- **Tools**: Ferramentas para cálculos
- **Chain of Thought**: Raciocínio passo a passo

In [ ]:
# Instalação das dependências
# !pip install langchain langchain-openai faiss-cpu python-dotenv

In [ ]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
from langchain.schema import Document
import math
from dotenv import load_dotenv

load_dotenv()

## 1. Criando a Base de Conhecimento (RAG)

Vamos criar documentos sobre física e matemática para nosso RAG:

In [ ]:
# Documentos de exemplo para RAG
documentos = [
    "A velocidade da luz no vácuo é 299.792.458 metros por segundo. É uma constante fundamental da física.",
    "A fórmula da energia cinética é E = (1/2) * m * v², onde m é a massa e v é a velocidade.",
    "A lei de Ohm estabelece que V = I * R, onde V é tensão, I é corrente e R é resistência.",
    "O teorema de Pitágoras diz que em um triângulo retângulo, a² + b² = c², onde c é a hipotenusa.",
    "A área de um círculo é π * r², onde r é o raio do círculo.",
    "A força gravitacional entre dois objetos é F = G * (m1 * m2) / r², onde G é a constante gravitacional."
]

# Converter strings em Documents
docs = [Document(page_content=doc) for doc in documentos]

# Dividir textos em chunks menores
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20
)
textos = text_splitter.split_documents(docs)

print(f"Criados {len(textos)} chunks de texto")

In [ ]:
# Criar embeddings e store vetorial
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(textos, embeddings)

print("Vectorstore criado com sucesso!")

## 2. Criando Tools (Ferramentas)

Vamos criar ferramentas para cálculos matemáticos:

In [ ]:
def calcular_energia_cinetica(massa: float, velocidade: float) -> float:
    """Calcula energia cinética usando E = (1/2) * m * v²"""
    return 0.5 * massa * (velocidade ** 2)

def calcular_area_circulo(raio: float) -> float:
    """Calcula área de um círculo usando π * r²"""
    return math.pi * (raio ** 2)

def teorema_pitagoras(a: float, b: float) -> float:
    """Calcula hipotenusa usando a² + b² = c²"""
    return math.sqrt(a**2 + b**2)

# Criar tools do LangChain
tool_energia = Tool(
    name="Calculadora_Energia_Cinetica",
    description="Calcula energia cinética. Use: 'massa,velocidade' (ex: '10,5' para 10kg e 5m/s)",
    func=lambda x: calcular_energia_cinetica(*[float(i) for i in x.split(',')])
)

tool_area = Tool(
    name="Calculadora_Area_Circulo",
    description="Calcula área de círculo. Use apenas o raio (ex: '5' para raio=5)",
    func=lambda x: calcular_area_circulo(float(x))
)

tool_pitagoras = Tool(
    name="Teorema_Pitagoras",
    description="Calcula hipotenusa. Use: 'cateto1,cateto2' (ex: '3,4')",
    func=lambda x: teorema_pitagoras(*[float(i) for i in x.split(',')])
)

tools = [tool_energia, tool_area, tool_pitagoras]

## 3. Configurando RAG Chain

Vamos criar uma chain que pode buscar informações nos documentos:

In [ ]:
# Configurar LLM
llm = OpenAI(temperature=0.1)

# Criar retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

# Criar RAG chain
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

# Criar tool para RAG
rag_tool = Tool(
    name="Consulta_Base_Conhecimento",
    description="Busca informações na base de conhecimento sobre física e matemática",
    func=lambda x: rag_chain.run(x)
)

# Adicionar RAG tool à lista
all_tools = tools + [rag_tool]

## 4. Criando Agent com Chain of Thought

Vamos criar um agente que combina RAG, tools e raciocínio passo a passo:

In [ ]:
# Prompt que incentiva Chain of Thought (Zero Prompting)
system_prompt = """
Você é um assistente de física e matemática que sempre:
1. Primeiro busca informações relevantes na base de conhecimento
2. Explica seu raciocínio passo a passo
3. Usa ferramentas de cálculo quando necessário
4. Mostra os cálculos e resultados claramente

SEMPRE siga este processo de pensamento:
- Passo 1: Identificar o que precisa ser resolvido
- Passo 2: Buscar fórmulas ou conceitos relevantes
- Passo 3: Aplicar cálculos se necessário
- Passo 4: Verificar e explicar o resultado
"""

# Criar agent
agent = initialize_agent(
    tools=all_tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

print("Agent configurado com sucesso!")

## 5. Exemplos Práticos

Agora vamos testar nosso sistema completo:

In [ ]:
# Exemplo 1: Pergunta que requer RAG + Chain of Thought
pergunta1 = """
Explique a fórmula da energia cinética e calcule a energia cinética 
de um objeto de 5kg movendo-se a 10 m/s.
"""

print("=== EXEMPLO 1: RAG + CoT + Tool ===\n")
resultado1 = agent.run(system_prompt + pergunta1)
print(f"\nResultado: {resultado1}")

In [ ]:
# Exemplo 2: Problema geométrico
pergunta2 = """
Preciso calcular a hipotenusa de um triângulo retângulo com catetos 
de 6 e 8 metros. Primeiro me explique o teorema envolvido.
"""

print("\n=== EXEMPLO 2: Teorema de Pitágoras ===\n")
resultado2 = agent.run(system_prompt + pergunta2)
print(f"\nResultado: {resultado2}")

In [ ]:
# Exemplo 3: Problema de área
pergunta3 = """
Qual é a fórmula para calcular a área de um círculo e qual seria 
a área de um círculo com raio de 3 metros?
"""

print("\n=== EXEMPLO 3: Área do Círculo ===\n")
resultado3 = agent.run(system_prompt + pergunta3)
print(f"\nResultado: {resultado3}")

## 6. Análise dos Componentes

### Como funciona cada parte:

**RAG (Retrieval-Augmented Generation):**
- Busca informações relevantes nos documentos indexados
- Fornece contexto atualizado para o LLM
- Reduz alucinações ao basear respostas em dados reais

**Zero Prompting:**
- Não usa exemplos específicos no prompt
- Confia na capacidade natural do LLM de seguir instruções
- Mais flexível que few-shot prompting

**Tools:**
- Permitem cálculos precisos
- Estendem capacidades do LLM
- Fornecem resultados determinísticos

**Chain of Thought:**
- Força o modelo a explicar seu raciocínio
- Melhora a qualidade das respostas
- Torna o processo transparente e auditável

In [ ]:
# Teste final: Problema complexo combinando tudo
pergunta_complexa = """
Um físico precisa resolver um problema: ele tem um objeto de 2kg 
movendo-se a 8 m/s. Ele quer saber:
1. Qual é a energia cinética deste objeto?
2. Se este objeto estivesse em um círculo de raio igual à velocidade (8m), 
   qual seria a área deste círculo?
3. Explique as fórmulas utilizadas.
"""

print("\n=== TESTE COMPLEXO: Combinando todos os conceitos ===\n")
resultado_complexo = agent.run(system_prompt + pergunta_complexa)
print(f"\nResultado: {resultado_complexo}")